In [3]:
import matplotlib.pyplot as plt
import mne
import seaborn as sns
import torch
from braindecode import EEGClassifier
from sklearn.pipeline import make_pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit
import matplotlib.pyplot as plt
import pandas as pd
import os

from moabb.datasets import BNCI2014_001, BNCI2014_004
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import MotorImagery
from moabb.utils import setup_seed

from moabb_Private_Encoder_Thesis.moabb.evaluations import SubjectParamEvaluation

#from moabb.evaluations import SubjectParamEvaluation
from shallow import CollapsedShallowNet ,ShallowFBCSPNet
from shallowDict import ShallowPrivateTemporalDictNetSlow, ShallowPrivateSpatialDictNetSlow, ShallowPrivateCollapsedDictNetSlow, SubjectDicionaryFCNet

mne.set_log_level(False)

# Print Information PyTorch
print(f"Torch Version: {torch.__version__}")

# Set up GPU if it is there
cuda = torch.cuda.is_available()
device = "cuda" if cuda else "cpu"
print("GPU is", "AVAILABLE" if cuda else "NOT AVAILABLE")

seed = 100
setup_seed(seed)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameter

# learning rate 1e-4

# batch = 2^7
LEARNING_RATE = 0.0001  # parameter taken from Braindecode
WEIGHT_DECAY = 0  # parameter taken from Braindecode
BATCH_SIZE = 128  # parameter taken from BrainDecode
EPOCH = 2
PATIENCE = 100
fmin = 4
fmax = 100
tmin = 0
tmax = None


dataset = BNCI2014_001()
paradigm = MotorImagery(
    fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax
)

X, _, _ = paradigm.get_data(dataset=dataset)

subjects = dataset.subject_list
        

def make_classifier(module):
    clf = EEGClassifier(
        module=module,  
        module__n_chans=X.shape[1],  # number of input channels
        module__n_outputs=len(dataset.event_id),  # number of output classes
        module__n_times=X.shape[2],  # length of the input signal in time points
        optimizer=torch.optim.Adam,
        optimizer__lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        max_epochs=EPOCH,
        warm_start=True, #To keep training the model further for each fit instead of re-initializing
        train_split=ValidSplit(0.2, random_state=seed),
        device=device,
        callbacks=[
            #EarlyStopping(monitor="valid_loss", patience=PATIENCE),
            EpochScoring(
                scoring="accuracy", on_train=True, name="train_acc", lower_is_better=False
            ),
            EpochScoring(
                scoring="accuracy", on_train=False, name="valid_acc", lower_is_better=False
            ),
        ],
        verbose=1,
    )
    
    return clf


clf1 = make_classifier(SubjectDicionaryFCNet)

clf2 = make_classifier(CollapsedShallowNet)

clf3 = make_classifier(ShallowPrivateTemporalDictNetSlow)

clf4 = make_classifier(ShallowFBCSPNet)

clf5 = make_classifier(ShallowPrivateSpatialDictNetSlow)

clf6 = make_classifier(ShallowPrivateCollapsedDictNetSlow)

# Create a pipeline with the classifier
#pipes = { "ShallowPrivateSpatialDictNetSlow": make_pipeline(clf5), "ShallowPrivateCollapsedDictNetSlow": make_pipeline(clf6)}


pipes = { "SubjectDicionaryFCNet": make_pipeline(clf1), "CollapsedShallowNet": make_pipeline(clf2), "ShallowPrivateTemporalDictNetSlow": make_pipeline(clf3), "ShallowFBCSPNet": make_pipeline(clf4), "ShallowPrivateSpatialDictNetSlow": make_pipeline(clf5), "ShallowPrivateCollapsedDictNetSlow": make_pipeline(clf6)}


results_list = []
# Ensure the output directory exists
output_dir = f"./results_{seed}_{dataset.code}"
os.makedirs(output_dir, exist_ok=True)

# Modify plot and data saving within the loop
for pipe_name, pipe in pipes.items():
    unique_suffix = f"{pipe_name}_braindecode_example"
    
    evaluation = SubjectParamEvaluation(
        paradigm=paradigm,
        datasets=[dataset],
        suffix=unique_suffix,
        overwrite=True,
        return_epochs=True,
        random_state=seed,
        n_jobs=1,
        hdf5_path=f"{output_dir}/{pipe_name}",
        save_model=True
    )
    
    # Run the evaluation process for this pipeline
    results = evaluation.process({pipe_name: pipe})

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{output_dir}/{pipe_name}_results.csv", index=False)

    # Save individual bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=results_df, y="score", x="subject", palette="viridis")
    plt.title(f"Model Performance by Subject - {pipe_name}")
    plt.ylabel("Score")
    plt.xlabel("Subject")
    plt.savefig(f"{output_dir}/{pipe_name}_performance.png")
    plt.close()
    
    results_list.append(results_df)

# Concatenate all results
results_all = pd.concat(results_list)
results_all.to_csv(f"{output_dir}/all_results.csv", index=False)

# Save combined bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=results_all, x="subject", y="score", hue="pipeline", palette="viridis")
plt.title("Scores per Subject for Each Pipeline")
plt.xlabel("Subject")
plt.ylabel("Score")
plt.legend(title="Pipeline")
plt.savefig(f"{output_dir}/combined_performance.png")
plt.close()

Choosing from all possible events


Torch Version: 2.2.2
GPU is NOT AVAILABLE
We try to set the tensorflow seeds, but it seems that tensorflow is not installed. Please refer to `https://www.tensorflow.org/` to install if you need to use this deep learning module.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.1

  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.2617        1.3873       0.2826        1.4064  0.3282
      2       0.2266        1.4298       0.2065        1.5215  0.3227
      3       0.2266        1.4043       0.2500        1.4441  0.3110
      4       0.2344        1.3974       0.3804        1.3722  0.3108
      5       0.2578        1.3958       0.1630        1.3963  0.2937
      6       0.2422        1.3894       0.2609        1.3964  0.3103
      7       0.2461        1.3856       0.2283        1.3905  0.2955
      8       0.2305        1.4098       0.2065        1.4027  0.3021
      9       0.3438        1.3610       0.2609        1.3544  0.2885
     10       0.3008        1.3828       0.3478        1.3760  0.3165
     11       0.2773        1.4012       0.2826        1.3931  0.2988
     12       0.2539        1.3938       0.2609        1.3982  0.3017
     13       0.3008

BNCI2014-001-SubjectParam: 100%|██████████| 1/1 [00:39<00:00, 39.17s/it]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {e

...

BNCI2014-001-SubjectParam: 100%|██████████| 1/1 [01:21<00:00, 81.58s/it]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {e

  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.2617        1.3869       0.2065        1.4709  2.7410
      2       0.2148        1.4295       0.2609        1.4625  2.3416
      3       0.3164        1.3765       0.2391        1.3930  2.2811
      4       0.2305        1.3947       0.3152        1.3860  2.5246
      5       0.2461        1.3847       0.2283        1.3948  2.5251
      6       0.2617        1.3824       0.3152        1.3877  2.3391
      7       0.2383        1.3915       0.3152        1.3696  2.4945
      8       0.2344        1.4040       0.2717        1.4103  2.4039
      9       0.2617        1.4078       0.1957        1.3767  2.2804
     10       0.2734        1.3843       0.2391        1.3843  2.5873
     11       0.2539        1.3995       0.2391        1.4167  2.3924
     12       0.2617        1.3916       0.2500        1.3659  2.3619
     13       0.2695

BNCI2014-001-SubjectParam: 100%|██████████| 1/1 [01:22<00:00, 82.11s/it]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {e

  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.2734        1.3939       0.2500        1.4686  0.5284
      2       0.2422        1.4176       0.2609        1.4217  0.3172
      3       0.2617        1.3980       0.2717        1.3945  0.3055
      4       0.2812        1.3822       0.2283        1.3869  0.2995
      5       0.2266        1.3970       0.1957        1.4027  0.3166
      6       0.2656        1.3921       0.2717        1.4282  0.3439
      7       0.1875        1.4002       0.2609        1.3990  0.3355
      8       0.2344        1.3895       0.2826        1.3938  0.3488
      9       0.2383        1.3854       0.2174        1.4192  0.3682
     10       0.2461        1.3814       0.2935        1.3905  0.3005
     11       0.1484        1.4225       0.3043        1.3855  0.3125
     12       0.2617        1.3857       0.3478        1.3757  0.2981
     13       0.3086

BNCI2014-001-SubjectParam: 100%|██████████| 1/1 [00:40<00:00, 40.82s/it]
